# Quicken Quickbooks Upgrade Assignment
- Rob's solutions

# 

# Imports

#### Standard library imports

In [1]:
import sys
sys.path.append("../")

import os

#### Third party imports

In [2]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

#### Local application imports

In [3]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

# 

# Importing data

In [4]:
df = pd.read_csv('../pkg_dir/data/intuit.csv')

# 

# Data exploration

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          20000 non-null  int64  
 1   sex         20000 non-null  object 
 2   bizflag     20000 non-null  int64  
 3   zip         20000 non-null  int64  
 4   zip_bins    20000 non-null  int64  
 5   numords     20000 non-null  int64  
 6   dollars     20000 non-null  float64
 7   last        20000 non-null  int64  
 8   sincepurch  20000 non-null  int64  
 9   version1    20000 non-null  int64  
 10  owntaxprod  20000 non-null  int64  
 11  upgraded    20000 non-null  int64  
 12  res1        20000 non-null  int64  
dtypes: float64(1), int64(11), object(1)
memory usage: 2.0+ MB


In [6]:
df.describe()

,id,bizflag,zip,zip_bins,numords,dollars,last,sincepurch,version1,owntaxprod,upgraded,res1
count,20000.000000,20000.000000,20000.0000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.244400,54350.2613,10.499000,2.079450,93.581000,15.779650,15.751400,0.208250,0.027100,0.215600,0.048050
std,5773.647028,0.429742,32776.4059,5.766755,1.246361,82.601398,9.527331,10.020435,0.406067,0.162379,0.411248,0.213877
min,1.000000,0.000000,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,23451.0000,5.000000,1.000000,37.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,55402.5000,10.500000,2.000000,71.000000,15.000000,15.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,89018.7500,15.250000,3.000000,124.500000,24.000000,24.000000,0.000000,0.000000,0.000000,0.000000
max,20000.000000,1.000000,99999.0000,20.000000,5.000000,937.500000,36.000000,36.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
## Number of positive labels in results

mba263.tabulate(df['res1'])

,Name,Count,Frequency
0,0,19039,0.95195
1,1,961,0.04805


In [8]:
## Do all customers who upgraded have Quickbooks version 2?

df.groupby(
    [
        'upgraded',
        'version1',
    ]
).agg(
    cnt=('id', 'count')
)

cnt
upgraded version1       
0        0         11523
         1          4165
1        0          4312

# 

# Wrangling

#### Converting sex into dummy variables

In [9]:
df = pd.concat(
    [
        df,
        pd.get_dummies(df['sex'], prefix='sex', drop_first=True)
    ],
    axis=1
)

# 

# Modeling
- Models developed:
    - M1: Logistic regression
    - M2: 

## M1: Logistic regression

#### Model

In [10]:
dfm1 = df.copy()

In [11]:
## Defining label and features for the model

m1_l = ['res1']

m1_f = [
    # 'id',
    # 'sex',
    'sex_F',
    'sex_M',
    'sex_U',
    'bizflag',
    # 'zip',
    'zip_bins',
    'numords',
    'dollars',
    'last',
    'sincepurch',
    'version1',
    'owntaxprod',
    'upgraded',
]

In [12]:
m1 = mba263.logit(
    dfm1[m1_l],
    dfm1[m1_f]
)

Optimization terminated successfully.
         Current function value: 0.178707
         Iterations 8


#### Coefficients

In [13]:
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                        Mba263Logit Regression Results                        
==============================================================================
Dep. Variable:                   res1   No. Observations:                20000
Model:                    Mba263Logit   Df Residuals:                    19987
Method:                           MLE   Df Model:                           12
Date:                Fri, 17 Mar 2023   Pseudo R-squ.:                 0.07278
Time:                        22:05:30   Log-Likelihood:                -3574.1
converged:                       True   LL-Null:                       -3854.7
Covariance Type:            nonrobust   LLR p-value:                2.165e-112
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1723      0.619     -3.512      0.000      -3.385      -0.960
sex_F         -0.7015      0.613     -1.144      0.253      -1.904       0.501
sex_M         -0.5670      0.609     -0.930      0.352      -1.762       0.628
sex_U         -0.6022      0.617     -0.975      0.329      -1.812       0.608
bizflag       -0.1160      0.086     -1.351      0.177      -0.284       0.052
zip_bins      -0.0575      0.006     -9.559      0.000      -0.069      -0.046
numords        0.1996      0.030      6.656      0.000       0.141       0.258
dollars        0.0010      0.000      2.547      0.011       0.000       0.002
last          -0.0499      0.004    -12.708      0.000      -0.058      -0.042
sincepurch     0.0072      0.006      1.146      0.252      -0.005       0.020
version1       0.6196      0.138      4.489      0.000       0.349       0.890
owntaxprod     0.0435      0.184      0.236      0.814      -0.318       0.405
upgraded       0.7524      0.135      5.575      0.000       0.488       1.017
==============================================================================
"""

In [14]:
mba263.odds_ratios(m1)

,Odds ratios,std err,z,P>|z|,[0.025,0.975]
sex_F,0.495844,0.304126,1.657723,0.097,-0.094160,1.085848
sex_M,0.567209,0.345712,1.251885,0.211,-0.103473,1.237890
sex_U,0.547582,0.338092,1.338151,0.181,-0.108317,1.203480
bizflag,0.890497,0.076430,1.432720,0.152,0.742222,1.038771
zip_bins,0.944158,0.005676,9.838666,0.000,0.933147,0.955169
numords,1.220947,0.036620,6.033489,0.000,1.149904,1.291989
dollars,1.001048,0.000412,2.546071,0.011,1.000249,1.001847
last,0.951346,0.003734,13.030364,0.000,0.944102,0.958590
sincepurch,1.007236,0.006339,1.141456,0.254,0.994938,1.019535
version1,1.858156,0.256472,3.345998,0.001,1.360599,2.355712


#### Predictions

In [15]:
dfm1['m1_pred_prob'] = m1.predict(exog=dfm1[m1_f])

###### 

## M2: Neural network

#### Model

In [16]:
dfm2 = df.copy()

In [17]:
## Defining label and features for the model

m2_l = ['res1']

m2_f = [
    # 'id',
    # 'sex',
    'sex_F',
    'sex_M',
    'sex_U',
    'bizflag',
    # 'zip',
    'zip_bins',
    'numords',
    'dollars',
    'last',
    'sincepurch',
    'version1',
    'owntaxprod',
    'upgraded',
]

In [18]:
m2 = mba263.neural_network(
    dfm1[m1_l],
    dfm1[m1_f]
)

#### Coefficients

#### Predictions

In [19]:
dfm2['m2_pred_prob'] = m2.predict(dfm1[m1_f])

###### 

## M3: Logistic regression with regularization

# 

# Results exploration

### Probability predictions distribution based on results

##### Model 1: Logistic regression

In [ ]:
fig = px.box(dfm1, x="res1", y=round(dfm1['m1_pred_prob'], 4))
fig.show()

In [ ]:
fig = px.histogram(
    dfm1, 
    x="m1_pred_prob", 
    color="res1", 
    nbins=200,
    marginal='rug',
    histnorm='probability',
    barmode='overlay'
)

fig.show()

##### Model 2: Neural network

In [ ]:
fig = px.box(dfm2, x="res1", y=round(dfm2['m2_pred_prob'], 4))
fig.show()

In [ ]:
fig = px.histogram(
    dfm2, 
    x="m2_pred_prob", 
    color="res1", 
    nbins=200,
    marginal='rug',
    histnorm='probability',
    barmode='overlay'
)

fig.show()

### Gains

In [ ]:
m1_gain = mba263.gain(dfm1['res1'], dfm1['m1_pred_prob'], bins=30)
m2_gain = mba263.gain(dfm2['res1'], dfm2['m2_pred_prob'], bins=30)

In [ ]:
plt.plot(m1_gain)
plt.plot(m2_gain)
plt.plot([0,30],[0,1],'b')

### Profit estimation
Calculate the best threshold to define a category and estimate values accordingly

# 

# *Notes*

#### Validating the break even response

In [21]:
60*100*0.0235 - 100*1.41

0.0

#### Validating the income generated by first wave

In [ ]:
cust_tot = 801821

#### Plotly express histogram documentation

histnorm: str (default `None`)
    One of `'percent'`, `'probability'`, `'density'`, or `'probability
    density'` If `None`, the output of `histfunc` is used as is. If
    `'probability'`, the output of `histfunc` for a given bin is divided by
    the sum of the output of `histfunc` for all bins. If `'percent'`, the
    output of `histfunc` for a given bin is divided by the sum of the
    output of `histfunc` for all bins and multiplied by 100. If
    `'density'`, the output of `histfunc` for a given bin is divided by the
    size of the bin. If `'probability density'`, the output of `histfunc`
    for a given bin is normalized such that it corresponds to the
    probability that a random event whose distribution is described by the
    output of `histfunc` will fall into that bin.

---

---